# TensorFlow hands on exercise: debugging a CNN

This segment will introduce a few more practical concepts necessary when implementing and debugging machine learning models in TensorFlow.
 1. TensorBoard tool for debugging and visualization
 1. Name scopes (variables, palceholders, ops..)
 1. Summary variables, histograms
 1. Hyperparameter search
 1. From layers to estimator API

Import **TensorFlow** and set up the log directory:

In [1]:
import os
import tensorflow as tf

LOGDIR = '/tmp/tutorial/'

## Dataset used for exercises: MNIST

To focus on algorithms, we are going to use the MNIST dataset here (we will explore some other datasets later).
MNIST contains greyscale images of 28x28 pixels size

<img src="images/mnist1.png">

<img src="images/mnist2.png">


MNIST dataset is available on the web. We are going to use an already normalized version from `tf.contrib.learn.datasets.mnist`:

In [2]:
mnist = tf.contrib.learn.datasets.mnist.read_data_sets(train_dir=LOGDIR + 'data', one_hot=True)

Extracting /tmp/tutorial/data/train-images-idx3-ubyte.gz
Extracting /tmp/tutorial/data/train-labels-idx1-ubyte.gz
Extracting /tmp/tutorial/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/tutorial/data/t10k-labels-idx1-ubyte.gz


# TensorFlow APIs

<img src="images/TF_api.png">

We will start by using layers API, then switch to estimators (Keras in particular).

# Fully connected layer

Layers will capture the logic of forward propagation step.

To define a fully connected (also known as dense) layer using the lowest level API in Tensorflow one could create
a function taking the input tensor (multidimensional array), input and output dimensions:

In [3]:
def fc_layer(input, size_in, size_out):
    w = tf.Variable(tf.zeros([size_in, size_out]))
    b = tf.Variable(tf.zeros([size_out]))
    act = tf.nn.relu(tf.matmul(input, w) + b)
    return act

The body of the function would contain a variable for the weight matrix and vector variable for the bias unit. As per forward propagation equations, the input tensor is multiplied by the weight matrix and the bias vector is added, after that the non-linearity is applied (`reLU` in this case).

## Hyperparameters

Type of variable initialization and non-linearity can be treated as hyperparameters and selected according to an optimization procedure.

Later, we will see the importance of variable initialization on the model convergence!

## Convolutional layer (2D)

Here, we are going to use `conv2d` layer implementation from TensorFlow:

```
tf.nn.conv2d

conv2d(
    input,
    filter,
    strides,
    padding,
    use_cudnn_on_gpu=None,
    data_format=None,
    name=None
)
```

Given an input tensor of shape [batch size, input height, input width, number of input channels] and a filter / kernel tensor of shape [filter height, filter width, number of input channels, number of output channels], this op performs the following:

 1. Flattens the filter to a 2-D matrix with shape [filter_height * filter_width * in_channels, output_channels].
 1. Extracts image patches from the input tensor to form a virtual tensor of shape [batch, out_height, out_width, filter_height * filter_width * in_channels].
 1. For each patch, right-multiplies the filter matrix and the image patch vector.

In [7]:
def conv_layer(input, size_in, size_out):
    w = tf.Variable(tf.zeros([5, 5, size_in, size_out]))
    b = tf.Variable(tf.zeros([size_out]))
    conv = tf.nn.conv2d(input, w, strides=[1, 1, 1, 1], padding="SAME")
    act = tf.nn.relu(conv + b)
    return tf.nn.max_pool(act, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

The pooling and convolutional ops slide a "window" across the input tensor. If the input tensor has 4 dimensions: *[batch, height, width, channels]*, then the convolution operates on a 2D window on the *height*, *width* dimensions.

`strides` argument determines how much the window shifts by in each of the dimensions. The typical use sets the first (the batch) and last (the depth) stride to 1. 

In our case we use the stride of 1 for heigh and depth in case of convolution and 2 for pooling.

<img src="images/pooling.png">

The `padding` parameter will determine the strategy oh how to deal with inconsistent dimensions. For instance:

```
    "VALID" = without padding:

       inputs:         1  2  3  4  5  6  7  8  9  10 11 (12 13)
                      |________________|                dropped
                                     |_________________|

    "SAME" = with zero padding:

                   pad|                                      |pad
       inputs:      0 |1  2  3  4  5  6  7  8  9  10 11 12 13|0  0
                   |________________|
                                  |_________________|
                                                 |________________|
```

Create session and reset the default graph:

In [8]:
tf.reset_default_graph()
sess = tf.Session()

Setup placeholders to read input data into, and reshape the data.

First, we take input tensor and reshape it Nx28x28x1, here 1 is the number of channels for greyscale image:

In [9]:
#the shape would be batch_size, 28x28=784
x = tf.placeholder(tf.float32, shape=[None, 784])

#second argument of reshape specifies the target shape
#-1 can be used to infer the shape - in this case, it is batch size
x_image = tf.reshape(x, [-1, 28, 28, 1])
tf.summary.image('input', x_image, 3)


the output shape would be batch_size x 10 labels (note, that for placeholders an unknown dimension is specified with None):

In [4]:
# the output shape would be batch_size, 10 labels
y = tf.placeholder(tf.float32, shape=[None, 10])

We then build a convolutional layer with 32 filters having spatial extent of 5 (both height and width). This is then passed through a max pooling layer which compresses the information.

We then build a second convolutional layer with 64 filters, again with spatial extent
5, taking an input tensor of depth 32 and emitting an output tensor of depth 64. Which is again passed through a pooling layer to compress information:

In [10]:
#Convolutional layers
conv1 = conv_layer(x_image, 1, 32)
conv_out = conv_layer(conv1, 32, 64)

We then prepare to pass the output of the max pooling layer into a
fully connected layer. To do this, we flatten the tensor. 

We have 64 filters, which corresponds to the tensor depth of 64. We now
have to determine the height and width after passing through two
max pooling layers. Using the following formulas:


$$ w_{out} = \frac{w_{in}-e+2\cdot p}{s}+1 = \frac{28-5+0}{1}+1 = 24, $$
$$ h_{out} = \frac{h_{in}-e+2\cdot p}{s}+1 = \frac{28-5+0}{1}+1 = 24, $$


Here, e denotes the spatial extent of the filter, s is the stride and p is their zero padding. 

For pooling:

$$ w_{out} = \frac{w_{in}-e}{s}+1 = \frac{24-2}{2}+1 = 7, $$
$$ h_{out} = \frac{h_{in}-e}{s}+1 = \frac{24-2}{2}+1 = 7 $$

So we get depth 64, height and width of 7:

In [ ]:
flattened = tf.reshape(conv_out, [-1, 7 * 7 * 64])

#Fully connected layers
fc1 = fc_layer(flattened, 7 * 7 * 64, 1024)
logits = fc_layer(fc1, 1024, 10)

Once the layers are defined, we need to define the op for loss calculation, optimizer and accuracy.

For this type of problem, we need to use softmax cross entropy.
It measures the probability error in discrete classification tasks in which the classes are mutually exclusive (each entry is in exactly one class).

In [11]:
#Calculate crossentropy loss
loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))

#Set up optimizer, use a dummy value for learning rate
train_step = tf.train.AdamOptimizer(1E-4).minimize(loss)

#accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Important step that every TensorFlow program will have is the variable initialization:

In [12]:
sess.run(tf.global_variables_initializer())

Set up summary writer and saver: so far, we do not have much summary statistics to aggregate, but we will add as we go.

In [14]:
summ = tf.summary.merge_all()
saver = tf.train.Saver()
writer = tf.summary.FileWriter(LOGDIR + "0")
writer.add_graph(sess.graph)

We will train the model iteratively, passing multiple times over the dataset. Each pass over a dataset is 
referred to as **epoch**. Since the dataset can be large and not fit into memory, the data is fed to the model in chunks or **mini-batches**, this way it can also be parallelized easier.

The mini-batches is a hyperparameter. During the training stages, we would like to optimize the throughput and feed as large a mini-batches as possible. There is a trade-off between speed of training and accuracy - smaller mini-batches tend to give better accuracy, but normally there is a reasonable batch size >> 1 which can be picked still allowing for a model to converge well.

During the inference... - this is a whole another story! In the real world applications most of the time we are dealing with pre-trained models doing inference only. The objective here is low latency - provide a prediction as fast as possible. Thus, smaller mini-batches are preferable.

In [15]:
for i in range(201):
    print "Train on mini-batch #{}".format(i)
    batch = mnist.train.next_batch(100)
    if i % 5 == 0:
        [train_accuracy, s] = sess.run([accuracy, summ], feed_dict={x: batch[0], y: batch[1]})
        writer.add_summary(s, i)
    if i % 50 == 0:
        saver.save(sess, os.path.join(LOGDIR, "model.ckpt"), i)
    sess.run(train_step, feed_dict={x: batch[0], y: batch[1]})

Train on mini-batch #0
Train on mini-batch #1
Train on mini-batch #2
Train on mini-batch #3
Train on mini-batch #4
Train on mini-batch #5
Train on mini-batch #6
Train on mini-batch #7
Train on mini-batch #8
Train on mini-batch #9
Train on mini-batch #10
Train on mini-batch #11
Train on mini-batch #12
Train on mini-batch #13
Train on mini-batch #14
Train on mini-batch #15
Train on mini-batch #16
Train on mini-batch #17
Train on mini-batch #18
Train on mini-batch #19
Train on mini-batch #20
Train on mini-batch #21
Train on mini-batch #22
Train on mini-batch #23
Train on mini-batch #24
Train on mini-batch #25
Train on mini-batch #26
Train on mini-batch #27
Train on mini-batch #28
Train on mini-batch #29
Train on mini-batch #30
Train on mini-batch #31
Train on mini-batch #32
Train on mini-batch #33
Train on mini-batch #34
Train on mini-batch #35
Train on mini-batch #36
Train on mini-batch #37
Train on mini-batch #38
Train on mini-batch #39
Train on mini-batch #40
Train on mini-batch #41
Tr

Putting it all together in a function (the calculation will have run as you evaluate the previous cell, but for later we will make changes directly in the body of `mnist_model` function):

In [16]:
def mnist_model(learning_rate, subfolder):
    tf.reset_default_graph()
    sess = tf.Session()

    # Setup placeholders, and reshape the data
    x = tf.placeholder(tf.float32, shape=[None, 784])
    x_image = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('input', x_image, 3)
    y = tf.placeholder(tf.float32, shape=[None, 10])

    #Convolutional layers
    conv1 = conv_layer(x_image, 1, 32)
    conv_out = conv_layer(conv1, 32, 64)

    flattened = tf.reshape(conv_out, [-1, 7 * 7 * 64])

    #Fully connected layers
    fc1 = fc_layer(flattened, 7 * 7 * 64, 1024)
    logits = fc_layer(fc1, 1024, 10)

    #Calculate crossentropy
    xent = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(
            logits=logits, labels=y))

    #Set up optimizer
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(xent)

    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    summ = tf.summary.merge_all()

    saver = tf.train.Saver()

    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(LOGDIR + subfolder)
    writer.add_graph(sess.graph)

    for i in range(201):
        print "Run epoch {}".format(i)
        batch = mnist.train.next_batch(100)
        if i % 5 == 0:
            [train_accuracy, s] = sess.run([accuracy, summ], feed_dict={x: batch[0], y: batch[1]})
            writer.add_summary(s, i)
        if i % 50 == 0:
            saver.save(sess, os.path.join(LOGDIR, "model.ckpt"), i)
        sess.run(train_step, feed_dict={x: batch[0], y: batch[1]})

Next, open the terminal window, and launch the TensorBoard:
```bash
python -m tensorflow.tensorboard --logdir=/tmp/tutorial/0
```
We are going to run a number of tests (5 tests in total), saving output for each in a separate subfolder.
Overall, you will have to point Tensorboard to your LOGDIR plus the test number, whatever you have set it to.

### What to expect

What we will see is only the flow graph. Unexpectly, the flow graph will be very messy and hard to comprehend. Why so complicated? We have a neural network constructed of convolutional and fully connected layers (simply enough), while I see 15-20 different boxes!?

### Solution: use name scopes

Let us go back to the layer definitions and pass a name scope parameter, keeping the rest the same:

```python
with tf.name_scope(name):
```

In addition, add specific name to each variable (as well as placeholders and ops), e.g.:
```python
        w = tf.Variable(tf.zeros([5, 5, size_in, size_out]), name="W")
        b = tf.Variable(tf.zeros([size_out]), name="B")
```        

In [19]:
def conv_layer(input, size_in, size_out, name="conv"):
    with tf.name_scope(name):
        w = tf.Variable(tf.zeros([5, 5, size_in, size_out]), name="W")
        b = tf.Variable(tf.zeros([size_out]), name="B")
        conv = tf.nn.conv2d(input, w, strides=[1, 1, 1, 1], padding="SAME")
        act = tf.nn.relu(conv + b)
        return tf.nn.max_pool(act, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

Repeat the same for the fully connected layer. Hint: you can use the same names for the same type of variables within the nested scope.

In [20]:
def fc_layer(input, size_in, size_out, name="fc"):
    with tf.name_scope(name):
        w = tf.Variable(tf.zeros([size_in, size_out]), name="W")
        b = tf.Variable(tf.zeros([size_out]), name="B")
        act = tf.nn.relu(tf.matmul(input, w) + b)
        return act

Analyze the results in the TensorBoard

In [21]:
def mnist_model(learning_rate, subdir):
    tf.reset_default_graph()
    sess = tf.Session()

    # Setup placeholders, and reshape the data
    x = tf.placeholder(tf.float32, shape=[None, 784], name="x")
    x_image = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('input', x_image, 3)
    y = tf.placeholder(tf.float32, shape=[None, 10], name="labels")

    conv1 = conv_layer(x_image, 1, 32, "conv1")
    conv_out = conv_layer(conv1, 32, 64, "conv2")

    flattened = tf.reshape(conv_out, [-1, 7 * 7 * 64])

    fc1 = fc_layer(flattened, 7 * 7 * 64, 1024, "fc1")
    logits = fc_layer(fc1, 1024, 10, "fc2")

    with tf.name_scope("xent"):
        xent = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y), name="xent")

    with tf.name_scope("train"):
        train_step = tf.train.AdamOptimizer(learning_rate).minimize(xent)

    with tf.name_scope("accuracy"):
        correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    summ = tf.summary.merge_all()

    saver = tf.train.Saver()

    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(LOGDIR + subdir) 
    writer.add_graph(sess.graph)

    for i in range(201):
        print "Train on mini batch # {}".format(i)
        batch = mnist.train.next_batch(100)
        if i % 5 == 0:
            [train_accuracy, s] = sess.run([accuracy, summ], feed_dict={x: batch[0], y: batch[1]})
            writer.add_summary(s, i)
        if i % 50 == 0:
            saver.save(sess, os.path.join(LOGDIR, "model.ckpt"), i)
        sess.run(train_step, feed_dict={x: batch[0], y: batch[1]})

Run it, pointing to a different subfolder:

In [22]:
def main(subdir):
    learning_rate = 1E-4
    mnist_model(learning_rate, subdir)

In [23]:
subdir = "1"
main(subdir)

Run epoch 0
Run epoch 1
Run epoch 2
Run epoch 3
Run epoch 4
Run epoch 5
Run epoch 6
Run epoch 7
Run epoch 8
Run epoch 9
Run epoch 10
Run epoch 11
Run epoch 12
Run epoch 13
Run epoch 14
Run epoch 15
Run epoch 16
Run epoch 17
Run epoch 18
Run epoch 19
Run epoch 20
Run epoch 21
Run epoch 22
Run epoch 23
Run epoch 24
Run epoch 25
Run epoch 26
Run epoch 27
Run epoch 28
Run epoch 29
Run epoch 30
Run epoch 31
Run epoch 32
Run epoch 33
Run epoch 34
Run epoch 35
Run epoch 36
Run epoch 37
Run epoch 38
Run epoch 39
Run epoch 40
Run epoch 41
Run epoch 42
Run epoch 43
Run epoch 44
Run epoch 45
Run epoch 46
Run epoch 47
Run epoch 48
Run epoch 49
Run epoch 50
Run epoch 51
Run epoch 52
Run epoch 53
Run epoch 54
Run epoch 55
Run epoch 56
Run epoch 57
Run epoch 58
Run epoch 59
Run epoch 60
Run epoch 61
Run epoch 62
Run epoch 63
Run epoch 64
Run epoch 65
Run epoch 66
Run epoch 67
Run epoch 68
Run epoch 69
Run epoch 70
Run epoch 71
Run epoch 72
Run epoch 73
Run epoch 74
Run epoch 75
Run epoch 76
Run epoch

After the calculation has finished, restart the TensorBoard pointing to `/tmp/tutorial/1` this time.

### What to expect

All tabs but Graph are still empty, but you can see a nice and clear flow graph at this time!
We cannot say anything else conclusive about the model performance yet. We can go and add some print statements 
or... let us use scalar variable summaries!

### Solution: scalar variable summaries

TensorFlow `tf.summary` class has `histogram` object:

```
histogram(
    name,
    values,
    collections=None
)
```

Here, `name` is the name scope. I.e. you can specify which variable to histogram by it's name. `values` is the reference to variable or op. But wait, can I specify a range, binning??? Let's wait and see.

In [26]:
def conv_layer(input, size_in, size_out, name="conv"):
    with tf.name_scope(name):
        w = tf.Variable(tf.zeros([5, 5, size_in, size_out]), name="W")
        b = tf.Variable(tf.zeros([size_out]), name="B")
        conv = tf.nn.conv2d(input, w, strides=[1, 1, 1, 1], padding="SAME")
        act = tf.nn.relu(conv + b)
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        return tf.nn.max_pool(act, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")


def fc_layer(input, size_in, size_out, name="fc"):
    with tf.name_scope(name):
        w = tf.Variable(tf.zeros([size_in, size_out]), name="W")
        b = tf.Variable(tf.zeros([size_out]), name="B")
        act = tf.nn.relu(tf.matmul(input, w) + b)
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        return act

In [27]:
def mnist_model(learning_rate, subdir):
    tf.reset_default_graph()
    sess = tf.Session()

    # Setup placeholders, and reshape the data
    x = tf.placeholder(tf.float32, shape=[None, 784], name="x")
    x_image = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('input', x_image, 3)
    y = tf.placeholder(tf.float32, shape=[None, 10], name="labels")

    conv1 = conv_layer(x_image, 1, 32, "conv1")
    conv_out = conv_layer(conv1, 32, 64, "conv2")

    flattened = tf.reshape(conv_out, [-1, 7 * 7 * 64])

    fc1 = fc_layer(flattened, 7 * 7 * 64, 1024, "fc1")
    logits = fc_layer(fc1, 1024, 10, "fc2")

    with tf.name_scope("xent"):
        xent = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y), name="xent")
        #Add histogram of loss
        tf.summary.scalar("xent", xent)

    with tf.name_scope("train"):
        train_step = tf.train.AdamOptimizer(learning_rate).minimize(xent)

    with tf.name_scope("accuracy"):
        correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        #Add histogram of accuracy
        tf.summary.scalar("accuracy", accuracy)

    summ = tf.summary.merge_all()

    saver = tf.train.Saver()

    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(LOGDIR + subdir)
    writer.add_graph(sess.graph)

    for i in range(201):
        print "Train on mini-batch # {}".format(i)
        batch = mnist.train.next_batch(100)
        if i % 5 == 0:
            [train_accuracy, s] = sess.run([accuracy, summ], feed_dict={x: batch[0], y: batch[1]})
            writer.add_summary(s, i)
        if i % 50 == 0:
            saver.save(sess, os.path.join(LOGDIR, "model.ckpt"), i)
        sess.run(train_step, feed_dict={x: batch[0], y: batch[1]})
        
def main(subdir):
    learning_rate = 1E-4
    mnist_model(learning_rate, subdir)        

Adjust the subfolder name and run it:

In [28]:
subdir = "2"
main(subdir)

Train on mini-batch # 0
Train on mini-batch # 1
Train on mini-batch # 2
Train on mini-batch # 3
Train on mini-batch # 4
Train on mini-batch # 5
Train on mini-batch # 6
Train on mini-batch # 7
Train on mini-batch # 8
Train on mini-batch # 9
Train on mini-batch # 10
Train on mini-batch # 11
Train on mini-batch # 12
Train on mini-batch # 13
Train on mini-batch # 14
Train on mini-batch # 15
Train on mini-batch # 16
Train on mini-batch # 17
Train on mini-batch # 18
Train on mini-batch # 19
Train on mini-batch # 20
Train on mini-batch # 21
Train on mini-batch # 22
Train on mini-batch # 23
Train on mini-batch # 24
Train on mini-batch # 25
Train on mini-batch # 26
Train on mini-batch # 27
Train on mini-batch # 28
Train on mini-batch # 29
Train on mini-batch # 30
Train on mini-batch # 31
Train on mini-batch # 32
Train on mini-batch # 33
Train on mini-batch # 34
Train on mini-batch # 35
Train on mini-batch # 36
Train on mini-batch # 37
Train on mini-batch # 38
Train on mini-batch # 39
Train on m

After the calculation has finished, restart the TensorBoard pointing to `/tmp/tutorial/2` this time.

### What to expect

Finally! Besides the "Graphs" we see the "Scalars", "Distributions" and "Histograms" tabs populated.

#### Scalars

This will have a distribution of loss (cross entropy) and the accuracy as a function of mini-batch number seen by the model. Accuracy is fluctuating around 10%, which is what we would get by random guessing (given 10 possible outcomes 0-9), and the loss is constant (which means no training!).

What went wrong?

(things are already wrong, so let us do not dig into histograms and distributions at this time)

### Solution: proper variable initialization

After carefully looking at the implementation of our convolutional and fully connected layers, we see that the initialization is unphysical: we initialize with zeros. Initialization scheme is a hyperparameter, possible options include normal, truncated normal, Glorot-style, etc.

Let us use truncated normal for weight matrix, and a constant (other than zero) for the bias vectors. Keep rest the same:

In [30]:
def conv_layer(input, size_in, size_out, name="conv"):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([5, 5, size_in, size_out], stddev=0.1), name="W")
        b = tf.Variable(tf.constant(0.1, shape=[size_out]), name="B")
        conv = tf.nn.conv2d(input, w, strides=[1, 1, 1, 1], padding="SAME")
        act = tf.nn.relu(conv + b)
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        return tf.nn.max_pool(act, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")


def fc_layer(input, size_in, size_out, name="fc"):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([size_in, size_out], stddev=0.1), name="W")
        b = tf.Variable(tf.constant(0.1, shape=[size_out]), name="B")
        act = tf.nn.relu(tf.matmul(input, w) + b)
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        return act

Increment output subfolder, and rerun (no changes to the main function at this time):

In [31]:
subdir = "3"
main(subdir)

Train on mini-batch # 0
Train on mini-batch # 1
Train on mini-batch # 2
Train on mini-batch # 3
Train on mini-batch # 4
Train on mini-batch # 5
Train on mini-batch # 6
Train on mini-batch # 7
Train on mini-batch # 8
Train on mini-batch # 9
Train on mini-batch # 10
Train on mini-batch # 11
Train on mini-batch # 12
Train on mini-batch # 13
Train on mini-batch # 14
Train on mini-batch # 15
Train on mini-batch # 16
Train on mini-batch # 17
Train on mini-batch # 18
Train on mini-batch # 19
Train on mini-batch # 20
Train on mini-batch # 21
Train on mini-batch # 22
Train on mini-batch # 23
Train on mini-batch # 24
Train on mini-batch # 25
Train on mini-batch # 26
Train on mini-batch # 27
Train on mini-batch # 28
Train on mini-batch # 29
Train on mini-batch # 30
Train on mini-batch # 31
Train on mini-batch # 32
Train on mini-batch # 33
Train on mini-batch # 34
Train on mini-batch # 35
Train on mini-batch # 36
Train on mini-batch # 37
Train on mini-batch # 38
Train on mini-batch # 39
Train on m

After the calculation has finished, restart the TensorBoard pointing to `/tmp/tutorial/3` this time.

### What to expect

Ok, we see that the loss is gradually decreasing as a function of the mini-batch seen so far. Model is learning something!

The accuracy is fluctuating between 15 and 20% - better than random guessing, but not quite what we are looking for...

Let us now look at the histograms tab:
 1. For every layer (we ahve 2 convolutional and 2 fully connected layers by construction), we get the distribution of values of weights, biases and activations as a function of `step` - mini-batch in our case 

the distributions do not change much from the initialization values - weight distribution in case of FC layers are close to truncated normal. Biases are close to constant in both cases. Activations: what did you expect to see here?
Weights: how many weights do we have, anyway?

Next, turn to distributions tab: same quiantities are shown. What type of plot is that?
This shows a set of bands superimposed, colorcoded. The central, darkmost layer is the mean values (as a function of step), next band corresponds to 1 sigma interval, 2 sigma and so on.


### Solution: hyperparameter search

Feels like we have tried just about everything, but the NN is not learning much... Let us step back and try to re-tune all those hyperparameters:
 1. Who said we should use 2 conv and 2 FC layers?
 1. Learning rate is the most important parameter to tune. Large learning rates can skip global minimum

Let us start with those 2 and maybe add more later... or maybe give up.

No changes to the layer functions this time. For the main function, we need a way to configure number of layers:

In [32]:
def mnist_model(learning_rate, use_two_conv, use_two_fc, hparam):
    tf.reset_default_graph()
    sess = tf.Session()

    # Setup placeholders, and reshape the data
    x = tf.placeholder(tf.float32, shape=[None, 784], name="x")
    x_image = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('input', x_image, 3)
    y = tf.placeholder(tf.float32, shape=[None, 10], name="labels")

    if use_two_conv:
        conv1 = conv_layer(x_image, 1, 32, "conv1")
        conv_out = conv_layer(conv1, 32, 64, "conv2")
    else:
        conv1 = conv_layer(x_image, 1, 64, "conv")
        conv_out = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

    flattened = tf.reshape(conv_out, [-1, 7 * 7 * 64])


    if use_two_fc:
        fc1 = fc_layer(flattened, 7 * 7 * 64, 1024, "fc1")
        logits = fc_layer(fc1, 1024, 10, "fc2")
    else:
        logits = fc_layer(flattened, 7*7*64, 10, "fc")

    with tf.name_scope("xent"):
        xent = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y), name="xent")
        tf.summary.scalar("xent", xent)

    with tf.name_scope("train"):
        train_step = tf.train.AdamOptimizer(learning_rate).minimize(xent)

    with tf.name_scope("accuracy"):
        correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        tf.summary.scalar("accuracy", accuracy)

    summ = tf.summary.merge_all()

    saver = tf.train.Saver()

    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(LOGDIR + hparam)
    writer.add_graph(sess.graph)


    for i in range(2001):
        print "Train on mini-batch # {}".format(i)
        batch = mnist.train.next_batch(100)
        if i % 5 == 0:
            [train_accuracy, s] = sess.run([accuracy, summ], feed_dict={x: batch[0], y: batch[1]})
            writer.add_summary(s, i)
        if i % 500 == 0:
            saver.save(sess, os.path.join(LOGDIR, "model.ckpt"), i)
        sess.run(train_step, feed_dict={x: batch[0], y: batch[1]})

To distinguish between configurations, we are going to need a helper-function:

In [33]:
def make_hparam_string(learning_rate, use_two_fc, use_two_conv):
    conv_param = "conv=2" if use_two_conv else "conv=1"
    fc_param = "fc=2" if use_two_fc else "fc=1"
    return "lr_%.0E,%s,%s" % (learning_rate, conv_param, fc_param)

We will do a simple grid search, so our main function will gain N for loop - one for each hyperparameter scanned (inefficient, I know...):

In [35]:
def main():
    # You can try adding some more learning rates
    for learning_rate in [1E-3, 1E-4, 1E-5]:

        # Include "False" as a value to try different model architectures
        for use_two_fc in [True, False]:
            for use_two_conv in [True, False]:
                # Construct a hyperparameter string for each one (example: "lr_1E-3,fc=2,conv=2)
                hparam = make_hparam_string(learning_rate, use_two_fc, use_two_conv)
                print('Starting run for %s' % hparam)

                # Actually run with the new settings
                mnist_model(learning_rate, use_two_fc, use_two_conv, hparam)

In [36]:
main()

Inside the main function
Starting run for lr_1E-03,conv=2,fc=2
Run epoch 0
Run epoch 1
Run epoch 2
Run epoch 3
Run epoch 4
Run epoch 5
Run epoch 6
Run epoch 7
Run epoch 8
Run epoch 9
Run epoch 10
Run epoch 11
Run epoch 12
Run epoch 13
Run epoch 14
Run epoch 15
Run epoch 16
Run epoch 17
Run epoch 18
Run epoch 19
Run epoch 20
Run epoch 21
Run epoch 22
Run epoch 23
Run epoch 24
Run epoch 25
Run epoch 26
Run epoch 27
Run epoch 28
Run epoch 29
Run epoch 30
Run epoch 31
Run epoch 32
Run epoch 33
Run epoch 34
Run epoch 35
Run epoch 36
Run epoch 37
Run epoch 38
Run epoch 39
Run epoch 40
Run epoch 41
Run epoch 42
Run epoch 43
Run epoch 44
Run epoch 45
Run epoch 46
Run epoch 47
Run epoch 48
Run epoch 49
Run epoch 50
Run epoch 51
Run epoch 52
Run epoch 53
Run epoch 54
Run epoch 55
Run epoch 56
Run epoch 57
Run epoch 58
Run epoch 59
Run epoch 60
Run epoch 61
Run epoch 62
Run epoch 63
Run epoch 64
Run epoch 65
Run epoch 66
Run epoch 67
Run epoch 68
Run epoch 69
Run epoch 70
Run epoch 71
Run epoch 7

Run epoch 589
Run epoch 590
Run epoch 591
Run epoch 592
Run epoch 593
Run epoch 594
Run epoch 595
Run epoch 596
Run epoch 597
Run epoch 598
Run epoch 599
Run epoch 600
Run epoch 601
Run epoch 602
Run epoch 603
Run epoch 604
Run epoch 605
Run epoch 606
Run epoch 607
Run epoch 608
Run epoch 609
Run epoch 610
Run epoch 611
Run epoch 612
Run epoch 613
Run epoch 614
Run epoch 615
Run epoch 616
Run epoch 617
Run epoch 618
Run epoch 619
Run epoch 620
Run epoch 621
Run epoch 622
Run epoch 623
Run epoch 624
Run epoch 625
Run epoch 626
Run epoch 627
Run epoch 628
Run epoch 629
Run epoch 630
Run epoch 631
Run epoch 632
Run epoch 633
Run epoch 634
Run epoch 635
Run epoch 636
Run epoch 637
Run epoch 638
Run epoch 639
Run epoch 640
Run epoch 641
Run epoch 642
Run epoch 643
Run epoch 644
Run epoch 645
Run epoch 646
Run epoch 647
Run epoch 648
Run epoch 649
Run epoch 650
Run epoch 651
Run epoch 652
Run epoch 653
Run epoch 654
Run epoch 655
Run epoch 656
Run epoch 657
Run epoch 658
Run epoch 659
Run ep

Run epoch 1163
Run epoch 1164
Run epoch 1165
Run epoch 1166
Run epoch 1167
Run epoch 1168
Run epoch 1169
Run epoch 1170
Run epoch 1171
Run epoch 1172
Run epoch 1173
Run epoch 1174
Run epoch 1175
Run epoch 1176
Run epoch 1177
Run epoch 1178
Run epoch 1179
Run epoch 1180
Run epoch 1181
Run epoch 1182
Run epoch 1183
Run epoch 1184
Run epoch 1185
Run epoch 1186
Run epoch 1187
Run epoch 1188
Run epoch 1189
Run epoch 1190
Run epoch 1191
Run epoch 1192
Run epoch 1193
Run epoch 1194
Run epoch 1195
Run epoch 1196
Run epoch 1197
Run epoch 1198
Run epoch 1199
Run epoch 1200
Run epoch 1201
Run epoch 1202
Run epoch 1203
Run epoch 1204
Run epoch 1205
Run epoch 1206
Run epoch 1207
Run epoch 1208
Run epoch 1209
Run epoch 1210
Run epoch 1211
Run epoch 1212
Run epoch 1213
Run epoch 1214
Run epoch 1215
Run epoch 1216
Run epoch 1217
Run epoch 1218
Run epoch 1219
Run epoch 1220
Run epoch 1221
Run epoch 1222
Run epoch 1223
Run epoch 1224
Run epoch 1225
Run epoch 1226
Run epoch 1227
Run epoch 1228
Run epoch 

Run epoch 1710
Run epoch 1711
Run epoch 1712
Run epoch 1713
Run epoch 1714
Run epoch 1715
Run epoch 1716
Run epoch 1717
Run epoch 1718
Run epoch 1719
Run epoch 1720
Run epoch 1721
Run epoch 1722
Run epoch 1723
Run epoch 1724
Run epoch 1725
Run epoch 1726
Run epoch 1727
Run epoch 1728
Run epoch 1729
Run epoch 1730
Run epoch 1731
Run epoch 1732
Run epoch 1733
Run epoch 1734
Run epoch 1735
Run epoch 1736
Run epoch 1737
Run epoch 1738
Run epoch 1739
Run epoch 1740
Run epoch 1741
Run epoch 1742
Run epoch 1743
Run epoch 1744
Run epoch 1745
Run epoch 1746
Run epoch 1747
Run epoch 1748
Run epoch 1749
Run epoch 1750
Run epoch 1751
Run epoch 1752
Run epoch 1753
Run epoch 1754
Run epoch 1755
Run epoch 1756
Run epoch 1757
Run epoch 1758
Run epoch 1759
Run epoch 1760
Run epoch 1761
Run epoch 1762
Run epoch 1763
Run epoch 1764
Run epoch 1765
Run epoch 1766
Run epoch 1767
Run epoch 1768
Run epoch 1769
Run epoch 1770
Run epoch 1771
Run epoch 1772
Run epoch 1773
Run epoch 1774
Run epoch 1775
Run epoch 

Run epoch 280
Run epoch 281
Run epoch 282
Run epoch 283
Run epoch 284
Run epoch 285
Run epoch 286
Run epoch 287
Run epoch 288
Run epoch 289
Run epoch 290
Run epoch 291
Run epoch 292
Run epoch 293
Run epoch 294
Run epoch 295
Run epoch 296
Run epoch 297
Run epoch 298
Run epoch 299
Run epoch 300
Run epoch 301
Run epoch 302
Run epoch 303
Run epoch 304
Run epoch 305
Run epoch 306
Run epoch 307
Run epoch 308
Run epoch 309
Run epoch 310
Run epoch 311
Run epoch 312
Run epoch 313
Run epoch 314
Run epoch 315
Run epoch 316
Run epoch 317
Run epoch 318
Run epoch 319
Run epoch 320
Run epoch 321
Run epoch 322
Run epoch 323
Run epoch 324
Run epoch 325
Run epoch 326
Run epoch 327
Run epoch 328
Run epoch 329
Run epoch 330
Run epoch 331
Run epoch 332
Run epoch 333
Run epoch 334
Run epoch 335
Run epoch 336
Run epoch 337
Run epoch 338
Run epoch 339
Run epoch 340
Run epoch 341
Run epoch 342
Run epoch 343
Run epoch 344
Run epoch 345
Run epoch 346
Run epoch 347
Run epoch 348
Run epoch 349
Run epoch 350
Run ep

...

Run epoch 1573
Run epoch 1574
Run epoch 1575
Run epoch 1576
Run epoch 1577
Run epoch 1578
Run epoch 1579
Run epoch 1580
Run epoch 1581
Run epoch 1582
Run epoch 1583
Run epoch 1584
Run epoch 1585
Run epoch 1586
Run epoch 1587
Run epoch 1588
Run epoch 1589
Run epoch 1590
Run epoch 1591
Run epoch 1592
Run epoch 1593
Run epoch 1594
Run epoch 1595
Run epoch 1596
Run epoch 1597
Run epoch 1598
Run epoch 1599
Run epoch 1600
Run epoch 1601
Run epoch 1602
Run epoch 1603
Run epoch 1604
Run epoch 1605
Run epoch 1606
Run epoch 1607
Run epoch 1608
Run epoch 1609
Run epoch 1610
Run epoch 1611
Run epoch 1612
Run epoch 1613
Run epoch 1614
Run epoch 1615
Run epoch 1616
Run epoch 1617
Run epoch 1618
Run epoch 1619
Run epoch 1620
Run epoch 1621
Run epoch 1622
Run epoch 1623
Run epoch 1624
Run epoch 1625
Run epoch 1626
Run epoch 1627
Run epoch 1628
Run epoch 1629
Run epoch 1630
Run epoch 1631
Run epoch 1632
Run epoch 1633
Run epoch 1634
Run epoch 1635
Run epoch 1636
Run epoch 1637
Run epoch 1638
Run epoch 

Run epoch 134
Run epoch 135
Run epoch 136
Run epoch 137
Run epoch 138
Run epoch 139
Run epoch 140
Run epoch 141
Run epoch 142
Run epoch 143
Run epoch 144
Run epoch 145
Run epoch 146
Run epoch 147
Run epoch 148
Run epoch 149
Run epoch 150
Run epoch 151
Run epoch 152
Run epoch 153
Run epoch 154
Run epoch 155
Run epoch 156
Run epoch 157
Run epoch 158
Run epoch 159
Run epoch 160
Run epoch 161
Run epoch 162
Run epoch 163
Run epoch 164
Run epoch 165
Run epoch 166
Run epoch 167
Run epoch 168
Run epoch 169
Run epoch 170
Run epoch 171
Run epoch 172
Run epoch 173
Run epoch 174
Run epoch 175
Run epoch 176
Run epoch 177
Run epoch 178
Run epoch 179
Run epoch 180
Run epoch 181
Run epoch 182
Run epoch 183
Run epoch 184
Run epoch 185
Run epoch 186
Run epoch 187
Run epoch 188
Run epoch 189
Run epoch 190
Run epoch 191
Run epoch 192
Run epoch 193
Run epoch 194
Run epoch 195
Run epoch 196
Run epoch 197
Run epoch 198
Run epoch 199
Run epoch 200
Run epoch 201
Run epoch 202
Run epoch 203
Run epoch 204
Run ep

Run epoch 721
Run epoch 722
Run epoch 723
Run epoch 724
Run epoch 725
Run epoch 726
Run epoch 727
Run epoch 728
Run epoch 729
Run epoch 730
Run epoch 731
Run epoch 732
Run epoch 733
Run epoch 734
Run epoch 735
Run epoch 736
Run epoch 737
Run epoch 738
Run epoch 739
Run epoch 740
Run epoch 741
Run epoch 742
Run epoch 743
Run epoch 744
Run epoch 745
Run epoch 746
Run epoch 747
Run epoch 748
Run epoch 749
Run epoch 750
Run epoch 751
Run epoch 752
Run epoch 753
Run epoch 754
Run epoch 755
Run epoch 756
Run epoch 757
Run epoch 758
Run epoch 759
Run epoch 760
Run epoch 761
Run epoch 762
Run epoch 763
Run epoch 764
Run epoch 765
Run epoch 766
Run epoch 767
Run epoch 768
Run epoch 769
Run epoch 770
Run epoch 771
Run epoch 772
Run epoch 773
Run epoch 774
Run epoch 775
Run epoch 776
Run epoch 777
Run epoch 778
Run epoch 779
Run epoch 780
Run epoch 781
Run epoch 782
Run epoch 783
Run epoch 784
Run epoch 785
Run epoch 786
Run epoch 787
Run epoch 788
Run epoch 789
Run epoch 790
Run epoch 791
Run ep

Run epoch 1286
Run epoch 1287
Run epoch 1288
Run epoch 1289
Run epoch 1290
Run epoch 1291
Run epoch 1292
Run epoch 1293
Run epoch 1294
Run epoch 1295
Run epoch 1296
Run epoch 1297
Run epoch 1298
Run epoch 1299
Run epoch 1300
Run epoch 1301
Run epoch 1302
Run epoch 1303
Run epoch 1304
Run epoch 1305
Run epoch 1306
Run epoch 1307
Run epoch 1308
Run epoch 1309
Run epoch 1310
Run epoch 1311
Run epoch 1312
Run epoch 1313
Run epoch 1314
Run epoch 1315
Run epoch 1316
Run epoch 1317
Run epoch 1318
Run epoch 1319
Run epoch 1320
Run epoch 1321
Run epoch 1322
Run epoch 1323
Run epoch 1324
Run epoch 1325
Run epoch 1326
Run epoch 1327
Run epoch 1328
Run epoch 1329
Run epoch 1330
Run epoch 1331
Run epoch 1332
Run epoch 1333
Run epoch 1334
Run epoch 1335
Run epoch 1336
Run epoch 1337
Run epoch 1338
Run epoch 1339
Run epoch 1340
Run epoch 1341
Run epoch 1342
Run epoch 1343
Run epoch 1344
Run epoch 1345
Run epoch 1346
Run epoch 1347
Run epoch 1348
Run epoch 1349
Run epoch 1350
Run epoch 1351
Run epoch 

Run epoch 1834
Run epoch 1835
Run epoch 1836
Run epoch 1837
Run epoch 1838
Run epoch 1839
Run epoch 1840
Run epoch 1841
Run epoch 1842
Run epoch 1843
Run epoch 1844
Run epoch 1845
Run epoch 1846
Run epoch 1847
Run epoch 1848
Run epoch 1849
Run epoch 1850
Run epoch 1851
Run epoch 1852
Run epoch 1853
Run epoch 1854
Run epoch 1855
Run epoch 1856
Run epoch 1857
Run epoch 1858
Run epoch 1859
Run epoch 1860
Run epoch 1861
Run epoch 1862
Run epoch 1863
Run epoch 1864
Run epoch 1865
Run epoch 1866
Run epoch 1867
Run epoch 1868
Run epoch 1869
Run epoch 1870
Run epoch 1871
Run epoch 1872
Run epoch 1873
Run epoch 1874
Run epoch 1875
Run epoch 1876
Run epoch 1877
Run epoch 1878
Run epoch 1879
Run epoch 1880
Run epoch 1881
Run epoch 1882
Run epoch 1883
Run epoch 1884
Run epoch 1885
Run epoch 1886
Run epoch 1887
Run epoch 1888
Run epoch 1889
Run epoch 1890
Run epoch 1891
Run epoch 1892
Run epoch 1893
Run epoch 1894
Run epoch 1895
Run epoch 1896
Run epoch 1897
Run epoch 1898
Run epoch 1899
Run epoch 

After the calculation has finished, restart the TensorBoard pointing to `/tmp/tutorial/` this time. This will contain a number of subfolders named after the hyperparameter set used. You can remove the subfolders from the previous tests for clarity: 
```bash
rm -rf /tmp/tutorial/{0,1,2,3}
```

Alternatively, you can uncheck those folders under `Runs` setion on the left.

### What to expect

Scalar distributions for loss and accuracy will now contain several curves superimposed.
Play with the smoothing parameter to remove the noise in the curves (typically, when plotting learning curves for each mini-batch we will observe a fair amount of fluctuations).

Let us determine the best and the worst combination (the best should give us 90% and the worst is around 10%), go to the distrbiutions tab and see the `Histograms` corresponding to those configurations. What can you say about it?

### Note on random hyperparameter search

A simple alternative to the grid search, is to provide the prior distributions for each hyperparameter. In our case:
 1. Layer configuration: categorical hyperparameter. There is only 4 configurations 1-2, 2-1, 1-1, 2-2. We probably want to explore all
 1. Learning rate: log-continuous hyperparameter
 
Define helper classes:

```python
class Hyperparam(object):

    @abc.abstractmethod
    def choice(self):
    	return 0

    def get_conf_entry(self,conf):
    	el = conf
    	for sub_path in self.path:
    		el = el[sub_path]
    	return el

    def assign_to_conf(self,conf,save_path):
    	val = self.choice()
    	print(" : ".join(self.path)+ ": {}".format(val))
    	el = conf
    	for sub_path in self.path[:-1]:
    		el = el[sub_path]
    	el[self.path[-1]] = val

    	with open(save_path+"changed_params.out", 'a+') as outfile:
    		for el in self.path:
	    		outfile.write("{} : ".format(el))
	    	outfile.write("{}\n".format(val))

class LogContinuousHyperparam(Hyperparam):
	def __init__(self,path,lo,hi):
		self.path = path
		self.lo = self.to_log(lo)
		self.hi = self.to_log(hi) 

	def to_log(self,num_val):
		return np.log10(num_val)

	def choice(self):
return float(np.power(10,np.random.uniform(self.lo,self.hi)))

class CategoricalHyperparam(Hyperparam):

	def __init__(self,path,values):
		self.path = path
		self.values = values

	def choice(self):
		return random.choice(self.values)

class GridCategoricalHyperparam(Hyperparam):

    def __init__(self,path,values):
        self.path = path
        self.values = iter(values)

    def choice(self):
        return next(self.values)
```

When doing random search, it is necessary to specify the number of trials. Each trial will train and score one configuration.

Next, let us talk moer about the evaluation metrics for how well the learning process is.
<p>Continue on to the next exercise: [6_Learning_and_Metrics.ipynb](6_Learning_and_Metrics.ipynb).</p>